In [342]:
import numpy as np
from collections import defaultdict
with open('./assets/input_day_12.txt','r') as file:
    elevation = np.array([list(iter(x)) for x in file.read().splitlines()])

class Graph():
    def __init__(self, elevation):
        """
        self.edges is a dict of all possible next nodes
        e.g. {'X': ['A', 'B', 'C', 'E'], ...}
        self.weights has all the weights between two nodes,
        with the two nodes as a tuple as the key
        e.g. {('X', 'A'): 7, ('X', 'B'): 2, ...}
        """
        self._elevation: np.array = np.array(elevation,copy=True)
        self.edges = defaultdict(list)
    
    def add_edge(self, from_node, to_node):
        # Note: assumes edges are bi-directional
        self.edges[from_node].append(to_node)
        # self.edges[to_node].append(from_node)

    def elevation(self,location):
        character = self._elevation[location]
        if character == 'S':
            return ord('a')
        if character == 'E':
            return ord('z')
        else:
            return ord(character)

    def get_allowed_neighbours(self,location: tuple):
        options = [(location[0]+1,location[1]),
        (location[0]-1,location[1]),
        (location[0],location[1]+1),
        (location[0],location[1]-1)
        ]

        options = [option for option in options if 
            not (option[0] < 0 
                or option[1] < 0 
                or option[0] >= self._elevation.shape[0] 
                or option[1] >= self._elevation.shape[1]) and (self.elevation(option) - self.elevation(location) <= 1)]

        return options

    def set_edges(self):
        for r in range(self._elevation.shape[0]):
            for c in range(self._elevation.shape[1]):
                options = self.get_allowed_neighbours((r,c))
                for option in options:
                    if option not in self.edges[(r,c)]:
                        self.add_edge((r,c),option)

    def reset_edges(self):
        self.edges = defaultdict(list)
        self.set_edges()


    def dijkstra(self, initial, end):
        # shortest paths is a dict of nodes
        # whose value is a tuple of (previous node, weight)
        shortest_paths = {initial: (None, 0)}
        current_node = initial
        visited = set()
        
        while current_node != end:
            visited.add(current_node)
            destinations = self.edges[current_node]
            weight_to_current_node = shortest_paths[current_node][1]

            for next_node in destinations:
                weight = 1 + weight_to_current_node
                if next_node not in shortest_paths:
                    shortest_paths[next_node] = (current_node, weight)
                else:
                    current_shortest_weight = shortest_paths[next_node][1]
                    if current_shortest_weight > weight:
                        shortest_paths[next_node] = (current_node, weight)
            
            next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}
            if not next_destinations:

                return False, [[v, self._elevation[v]] for v in visited]
            # next node is the destination with the lowest weight
            current_node = min(next_destinations, key=lambda k: next_destinations[k][1])
        
        # Work back through destinations in shortest path
        path = []
        while current_node is not None:
            path.append([current_node,self._elevation[current_node]])
            next_node = shortest_paths[current_node][0]
            current_node = next_node
        # Reverse path
        path = path[::-1]
        return True, path


In [343]:
graph = Graph(elevation)
graph.set_edges()
_, ds = graph.dijkstra((tuple(map(int,np.where(elevation == 'S')))),tuple(map(int,np.where(elevation == 'E'))))
print(len(ds)-1)

380


In [341]:
graph2 = Graph(elevation)
graph2.set_edges()
shortest = None

available_a = [(x,y) for x,y in zip(*np.where(np.isin(graph2._elevation,['a','S'])))]
while len(available_a) != 0:
    print(f"there are still {len(available_a)} starting points available")
    success ,ds = graph2.dijkstra(available_a[0],tuple(map(int,np.where(elevation == 'E')))) 
    a = [(i,x[0]) for i,x in enumerate(ds) if x[1] in ['a','S']]
    if success: 
        if shortest is None or len(ds[a[-1][0]:]) < len(shortest):
            shortest = ds[a[-1][0]:]

    for loc in [x[1] for x in a]:
        graph2._elevation[loc] = chr(ord('z')+100)

    available_a = [(x,y) for x,y in zip(*np.where(np.isin(graph2._elevation,['a','S'])))]

there are still 1339 starting points available
there are still 1336 starting points available
there are still 1297 starting points available
there are still 1276 starting points available
there are still 986 starting points available
there are still 949 starting points available
there are still 925 starting points available
there are still 890 starting points available
there are still 883 starting points available
there are still 877 starting points available
there are still 871 starting points available
there are still 865 starting points available
there are still 862 starting points available
there are still 837 starting points available
there are still 778 starting points available
there are still 746 starting points available
there are still 721 starting points available
there are still 720 starting points available
there are still 715 starting points available
there are still 504 starting points available
there are still 490 starting points available
there are still 489 starting p

In [340]:
len(shortest)-1

375